Imports

In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np

TF-IDF class. Contains methods which can easily access relevant information from the csv file which the class reads, which represents the TF-IDF model.

TF-IDF stands for Term Frequency - Inverse Document Frequency. It gives each keyword-document combination a weight, based on how often it occurs in the document, but also based on how often it occurs in all documents. This ensures that words which occur often in many documents do not receive high weights, rather only words which are relatively unique to a document and occur frequently in that document get high weights.

If all documents are about topic A, we do not care that document x contains is about document A.

Implementation wise, the TF-IDF is a csv file with each row representing a document and each column representing a keyword, the value at i,j represents the weight that keyword j has in document i.

In [2]:
class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("../data/tf_idf/tf_idf.csv")

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("../data/tf_idf/tf_idf_keywords.csv")
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id-1, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id-1][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

Instantiate the object

In [3]:
tfidf = TfIdf()

Read in the dataset of visits, used to test the information need prediction.

In [4]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('../data/clickdata/dataNoUnscrapedVisitsOrUnder20Sec.csv', converters=conv)

Create seperate dataframes for the paths taken and for the seconds spent on each url.
In paths, index 0 gives a list of urls, the equivalent index 0 in seconds represents how many seconds the user spent on the equal index urls.

In [5]:
paths = df.url_id_path
print(len(paths))

3583310


In [22]:
list = tfidf.get_all_keywords_by_id(1)
for i in range(0, len(list)):
    if list[i]!=0:
        print(i)

3534
8038
8983


)Simplified implementation of IUNIS, which simply looks at the keywords which the user comes across, does not use the adjacency matrix, proximal cues, and spreading activation.
It seems that these are mostly important for WUFIS, which we are not interested in, thus implementing it is not worth the effort.

Input is the ordered list of page id's that the user has visited.
For each page, take the TF-IDF values from that page, which is a row of keywords, with either 0 if the keyword is not present or [0,1] if it is.

Then, for each next page, we add the TF-IDF values onto the existing weights.

Eventually we can sort the list and we will have the keywords with the highest weight on top, which then represents the most relevant keywords in the visit up until now, and acts as our prediction for what the user is looking for.
Instead of only looking at the user alone, we can also compare their information need to information needs of other users, and by looking at where the other users end up, we can predict where our user might want to end up.

Optionally, a decay factor can be introdced. Essentially, every iteration, all values are divided by a factor, meaning that keywords on recent pages are more biased and weighted more heavily.

It is also possible to give more weight to pages based on how much time the user spent on that page.

In [8]:
from pandas import DataFrame
#Method which essentially just takes the keywords and their weights on each page the user visits, continues to the next url, and then sums the weights.
#Be sure to realise that only passing a path will execute the normal implementation, which simply sums all new tf-idf values of each url.
#This is because the default decay_factor value is 1, meaning that nothing changes, and the default secs is None, if that is None, the if statement with secs is never entered and we do not take time into account.
#Sorted gives an option to sort the vector before returning, this puts the most relevant keywords on top, but does not allow for easy similarity comparison, thus should only be used if this is not the intention
def find_keyword_weights(path, decay_factor=1, secs=None, sorted=False):
    #initiate array of weights (weights start as 0, size=num of keywords)
    weights = np.zeros(tfidf.get_number_of_keywords())
    #ensures all output is visible
    pd.options.display.max_rows = 0
    #Iterate over each page url
    for i in range(len(path)):
        #Divide the weights by the decay factor (1 by default, no decay in that case)
        weights = weights/decay_factor
        #Initialize the tf-idf weights of the current page
        new_weights = tfidf.get_all_keywords_by_id_normalized(path[i])
        #If we are taking time into account, multiply the weights by the amount of time spent on this page
        if secs is not None:
            new_weights*=secs[i]
        #Add the new weights to the existing weights
        weights += new_weights
        #Normalize such that the largest value becomes 1
        weights /= max(weights)
    #Option to sort s.t. max weights are on top.
    if sorted:
        df = DataFrame(weights, columns=['weights'])
        return df.sort_values(by=['weights'], ascending=False)
    return weights

Method which calculates keywords of a visit

In [12]:
def create_dictionary(keys, values):
    result = {} # empty dictionary
    for key, value in zip(keys, values):
        result[key] = value
    return result

In [18]:
def compute_keywords_of_complete_visit(min_visit_size, paths):
    keys = []
    for i in range(0, tfidf.get_number_of_keywords()):
        keys.append(tfidf.get_keyword_by_id(i))

    for path in paths:
        path[:] = [x for x in path if x<=1555]
        if len(path)>min_visit_size:
            weights = find_keyword_weights(path, decay_factor=7)
            keywords_weights = create_dictionary(keys, weights)
            print(keywords_weights)

In [19]:
compute_keywords_of_complete_visit(5, paths)

{'aa': 0.0, 'aaa': 0.0, 'aafje': 0.0, 'aambeien': 0.0, 'aan': 0.0, 'aanbesteding': 0.0, 'aanbeveling': 0.0, 'aanbevelingen': 0.0, 'aanbevolen': 0.0, 'aanbieden': 0.0, 'aanbieder': 0.0, 'aanbiedingen': 0.0, 'aanbiedt': 0.0, 'aanbod': 0.0, 'aanbouw': 0.0, 'aanbrengen': 0.0, 'aandacht': 0.0, 'aandachtpunten': 0.0, 'aandachtspunt': 0.0, 'aandeel': 0.0, 'aandeelhouder': 0.0, 'aandeelhouderschap': 0.0, 'aandeelhouderscollectief': 0.0, 'aandeelhoudersrechten': 0.0, 'aandeelhoudersseizoen': 0.0, 'aandeelhoudersstemmen': 0.0, 'aandeelhoudersvergadering': 0.0, 'aandeelhoudersvoorstel': 0.0, 'aandeelhouderwaarde': 0.0, 'aandelen': 0.0, 'aandelenbelang': 0.0, 'aandelenbeleggingen': 0.0, 'aandelenbeurzen': 0.0, 'aandelenkoersen': 0.0, 'aandelenmarkt': 0.0, 'aandelenportefeuille': 0.0, 'aandelenstrategie': 0.0, 'aandenken': 0.0, 'aandiende': 0.0, 'aandoening': 0.0, 'aandrijven': 0.0, 'aandrijving': 0.0, 'aandringen': 0.0, 'aaneengesloten': 0.0, 'aaneenschakeling': 0.0, 'aangaan': 0.0, 'aangaat': 0.0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

